# Word Embeddings via PMI Matrix Factorization

*Contact TA: emaad[at]cmu.edu, [eyeshalfclosed.com/teaching/](http://www.eyeshalfclosed.com/teaching/)*

   * Based on [Neural Word Embedding as Implicit Matrix Factorization](https://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization), by Omar Levy and Yoav Goldberg, NIPS 2014.
   * Dataset: https://www.kaggle.com/hacker-news/hacker-news-posts/downloads/HN_posts_year_to_Sep_26_2016.csv
   * Notes: http://www.eyeshalfclosed.com/teaching/95865-recitation-word2vec_as_PMI.pdf
   * Source material: https://multithreaded.stitchfix.com/blog/2017/10/18/stop-using-word2vec/.
   * Source material: https://www.kaggle.com/alexklibisz/simple-word-vectors-with-co-occurrence-pmi-and-svd

In [37]:
from collections import Counter
from itertools import combinations
from math import log
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pformat
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, norm
from string import punctuation

## 0. Load the data.

In [6]:
df = pd.read_csv('HN_posts_year_to_Sep_26_2016.csv', usecols=['title'])
df.head()

,title
0,You have two days to comment if you want stem ...
1,SQLAR the SQLite Archiver
2,What if we just printed a flatscreen televisio...
3,algorithmic music
4,How the Data Vault Enables the Next-Gen Data W...


## 1. Read and preprocess titles from HN posts.

In [10]:
%%time
punctrans = str.maketrans(dict.fromkeys(punctuation))
def tokenize(title):
    x = title.lower() # Lowercase
    x = x.encode('ascii', 'ignore').decode() # Keep only ascii chars.
    x = x.translate(punctrans) # Remove punctuation
    return x.split() # Return tokenized.

texts_tokenized = df['title'].apply(tokenize)

CPU times: user 1.11 s, sys: 69.8 ms, total: 1.18 s
Wall time: 1.19 s


## 2a. Compute unigram and bigram counts.

A unigram is a single word (x). A bigram is a pair of words (x,y).
Bigrams are counted for any two terms occurring in the same title.
For example, the title "Foo bar baz" has unigrams [foo, bar, baz]
and bigrams [(bar, foo), (bar, baz), (baz, foo)]

In [11]:
%%time
cx = Counter()
cxy = Counter()
for text in texts_tokenized:
    for x in text:
        cx[x] += 1
    for x, y in map(sorted, combinations(text, 2)):
        cxy[(x, y)] += 1

CPU times: user 13.2 s, sys: 317 ms, total: 13.5 s
Wall time: 14 s


## 2b. Remove frequent and infrequent unigrams.

Pick arbitrary occurrence count thresholds to eliminate unigrams occurring
very frequently or infrequently. This decreases the vocab size substantially.

In [13]:
%%time
print('%d tokens before' % len(cx))
min_count = (1 / 1000) * len(df)
max_count = (1 / 50) * len(df)
for x in list(cx.keys()):
    if cx[x] < min_count or cx[x] > max_count:
        del cx[x]
print('%d tokens after' % len(cx))
print('Most common:', cx.most_common()[:25])

99181 tokens before
1045 tokens after
Most common: [('i', 5577), ('google', 5532), ('be', 5320), ('app', 5124), ('as', 5121), ('its', 5077), ('about', 4927), ('can', 4801), ('using', 4613), ('do', 4534), ('not', 4330), ('us', 4189), ('web', 4134), ('will', 4125), ('we', 4113), ('startup', 3849), ('open', 3828), ('first', 3730), ('code', 3705), ('apple', 3695), ('pdf', 3659), ('more', 3652), ('software', 3558), ('my', 3515), ('this', 3477)]
CPU times: user 114 ms, sys: 4.37 ms, total: 118 ms
Wall time: 118 ms


## 2c. Remove frequent and infrequent bigrams.

Any bigram containing a unigram that was removed must now be removed.

In [15]:
%%time
for x, y in list(cxy.keys()):
    if x not in cx or y not in cx:
        del cxy[(x, y)]

CPU times: user 3.42 s, sys: 34.5 ms, total: 3.45 s
Wall time: 3.48 s


## 3. Build unigram <-> index lookup.

In [16]:
%%time
x2i, i2x = {}, {}
for i, x in enumerate(cx.keys()):
    x2i[x] = i
    i2x[i] = x

CPU times: user 558 µs, sys: 37 µs, total: 595 µs
Wall time: 602 µs


## 4. Sum unigram and bigram counts for computing probabilities.


In [17]:
sx = sum(cx.values())
sxy = sum(cxy.values())

# 5. Accumulate data, rows, and cols to build sparse PMI matrix

The PMI value for a bigram with tokens (x, y) is:
$$ \textrm{PMI}(x,y) = \frac{\textrm{log}(p(x,y))}{p(x)p(y)} $$

The probabilities are computed on the fly using the sums from above.

In [19]:
%%time
pmi_samples = Counter()
data, rows, cols = [], [], []
for (x, y), n in cxy.items():
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(log((n / sxy) / (cx[x] / sx) / (cx[y] / sx)))
    pmi_samples[(x, y)] = data[-1]
PMI = csc_matrix((data, (rows, cols)))
print('%d non-zero elements' % PMI.count_nonzero())
print('Sample PMI values\n', pformat(pmi_samples.most_common()[:10]))

297427 non-zero elements
Sample PMI values
 [(('elon', 'musk'), 6.8839034669891745),
 (('pi', 'raspberry'), 6.764144313961322),
 (('street', 'wall'), 6.6818495161706615),
 (('francisco', 'san'), 6.497633763166996),
 (('capital', 'venture'), 6.444633948553916),
 (('basic', 'income'), 6.329754881712322),
 (('card', 'credit'), 6.257880347803981),
 (('studio', 'visual'), 6.241828638793474),
 (('star', 'wars'), 6.176785106834259),
 (('command', 'line'), 6.125005172051617)]
CPU times: user 901 ms, sys: 38.5 ms, total: 940 ms
Wall time: 951 ms


## 6. Factorize the PMI matrix using sparse SVD aka "learn the unigram/word vectors".

This part replaces the stochastic gradient descent used by Word2vec
and other related neural network formulations. We pick an arbitrary vector size k=20.

In [21]:
%%time
U, _, _ = svds(PMI, k=20)

CPU times: user 248 ms, sys: 46.7 ms, total: 295 ms
Wall time: 389 ms


Normalize the vectors to compute cosine similarity.

In [22]:
norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
U /= np.maximum(norms, 1e-7)

## 8. Show some nearest neighbor samples as a sanity-check.

The format is `<unigram> <count>: (<neighbor unigram>, <similarity>), ...`
    
From this we can see that the relationships make sense.

In [25]:
k = 5
for x in ['facebook', 'twitter', 'instagram', 'messenger', 'hack', 'security', 
          'deep', 'encryption', 'cli', 'venture', 'paris']:
    dd = np.dot(U, U[x2i[x]]) # Cosine similarity for this unigram against all others.
    s = ''
    # Compile the list of nearest neighbor descriptions.
    # Argpartition is faster than argsort and meets our needs.
    for i in np.argpartition(-1 * dd, k + 1)[:k + 1]:
        if i2x[i] == x: continue
        s += '(%s, %.3lf) ' % (i2x[i], dd[i])
    print('%s, %d\n %s' % (x, cx[x], s))
    print('-' * 10)

facebook, 2853
 (ads, 0.804) (bot, 0.753) (google, 0.747) (instagram, 0.735) (app, 0.706) 
----------
twitter, 1641
 (tv, 0.822) (time, 0.810) (traffic, 0.782) (tracking, 0.772) (type, 0.765) 
----------
instagram, 391
 (facebook, 0.735) (links, 0.765) (news, 0.695) (images, 0.691) (search, 0.686) 
----------
messenger, 374
 (messaging, 0.764) (chat, 0.693) (lets, 0.633) (facebook, 0.612) (platform, 0.608) 
----------
hack, 881
 (hacked, 0.790) (hackers, 0.922) (hacking, 0.769) (malware, 0.695) (emails, 0.681) 
----------
security, 2425
 (software, 0.640) (secure, 0.718) (remote, 0.791) (push, 0.651) (process, 0.649) 
----------
deep, 1375
 (learning, 0.866) (neural, 0.852) (networks, 0.822) (algorithms, 0.820) (machine, 0.779) 
----------
encryption, 968
 (fbi, 0.849) (government, 0.836) (nsa, 0.739) (attacks, 0.739) (crypto, 0.911) 
----------
cli, 311
 (command, 0.877) (custom, 0.817) (easy, 0.786) (client, 0.764) (browser, 0.740) 
----------
venture, 393
 (where, 0.896) (university

## 9. Word-vector compositions

In [63]:
composition = U[x2i["facebook"]] - U[x2i["ads"]]
composition /= np.linalg.norm(composition)

k = 2
composition = U[x2i["facebook"]] + U[x2i["images"]]
dd = np.dot(U, composition) # Cosine similarity for this unigram against all others.
s = ''
for i in np.argpartition(-1 * dd, k + 1)[:k + 1]:
    s += '(%s, %.3lf) ' % (i2x[i], dd[i])
print(s)

(images, 1.460) (facebook, 1.460) (instagram, 1.426) 
